In [45]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from transformers import pipeline

# Load Aspect-Based Sentiment Analysis model
absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-large-absa-v1.1")
absa_model = AutoModelForSequenceClassification.from_pretrained("yangheng/deberta-v3-large-absa-v1.1")

# Load a traditional Sentiment Analysis model
sentiment_model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_model = pipeline("sentiment-analysis", model=sentiment_model_path,
                          tokenizer=sentiment_model_path)

In [46]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [52]:
# sentence = "We had a great experience at the restaurant, food was delicious, but the service was kinda bad"
# sentence = "China is laughing at BIden/Dems for showing them images of Russian troops on Ukraine border asking China for help instead of facing Russia like grown ups"
# sentence = "Biden is doing an amazing job with the Ukraine war. China should be ashamed of starting the war"
# sentence = "Joe Biden needs to call in President Trump to take care of this Putin Russian invasion of Ukraine as he is clearly not up to the task. And let him straighten out the border and inflation while hes at it. Win. Win. America is tired of losing because of Joe."
# sentence = "USA is sending support to Ukraine in the form of tanks and guns, as they should as they started the war"
# sentence = "USA is sending support to Ukraine in the form of tanks and guns, they started the war by meddling in our affairs"
# sentence = "I don't understand why POTUS is interferring in the war in Ukraine"
sentence = "I think POTUS helping Ukraine has been an awful mistake"
sentence = "Joe Biden needs to call in President Trump to take care of this Putin Russian invasion of Ukraine as he is clearly not up to the task. And let him straighten out the border and inflation while hes at it. Win. Win. America is tired of losing because of Joe."
print(f"Sentence: {sentence}")
print()

aspect = "Joe Biden"
inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
outputs = absa_model(**inputs)
probs = F.softmax(outputs.logits, dim=1)
probs = probs.detach().numpy()[0]
print(f"Sentiment of aspect '{aspect}' is:")
for prob, label in zip(probs, ["negative", "neutral", "positive"]):
  print(f"Label {label}: {prob}")
print()

aspect = "Trump"
inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
outputs = absa_model(**inputs)
probs = F.softmax(outputs.logits, dim=1)
probs = probs.detach().numpy()[0]
print(f"Sentiment of aspect '{aspect}' is:")
for prob, label in zip(probs, ["negative", "neutral", "positive"]):
  print(f"Label {label}: {prob}")
print()

# Overall sentiment of the sentence
sentiment = sentiment_model([sentence])[0]
print(f"Overall sentiment: {sentiment['label']} with score {sentiment['score']}")
# Overall sentiment: Negative with score 0.7706006765365601

Sentence: Joe Biden needs to call in President Trump to take care of this Putin Russian invasion of Ukraine as he is clearly not up to the task. And let him straighten out the border and inflation while hes at it. Win. Win. America is tired of losing because of Joe.

Sentiment of aspect 'Joe Biden' is:
Label negative: 0.9991238713264465
Label neutral: 0.0008288325625471771
Label positive: 4.728121712105349e-05

Sentiment of aspect 'Donald Trump' is:
Label negative: 0.049503300338983536
Label neutral: 0.6226859092712402
Label positive: 0.32781079411506653

Overall sentiment: negative with score 0.918430507183075
